In [12]:
import folium
import json

In [9]:
# Center on Montreal
m = folium.Map(location=[45.5, -73.6], zoom_start=10)

In [14]:
# Load JSON
with open("../data/QC-MONT-STD-SORT2-CASCADE-TSL.json") as f:
    data = json.load(f)

In [15]:
polygons = []

for route in data.get("routingParameterUiVehicleDTOs", []):
    for pref in route.get("routingParameterUiVehiclePreferenceDTOs", []):
        poly_data = pref.get("routingParameterUiPolygonDTO")
        if poly_data and "polygonCoordinates" in poly_data:
            coords_text = poly_data["polygonCoordinates"].strip()
            coords = []
            for line in coords_text.splitlines():
                lon, lat, *_ = line.split(",")
                coords.append((float(lat), float(lon)))  # folium expects (lat, lon)
            polygons.append({
                "name": poly_data.get("name", "Unknown"),
                "coords": coords
                   })

In [16]:
for poly in polygons:
    folium.Polygon(
        locations=poly["coords"],
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.2,
        tooltip=poly["name"]
        # tooltip=folium.GeoJsonTooltip(fields=poly["name"], aliases=["Name: "])
    ).add_to(m)

In [17]:
m.save("map.html")

In [2]:
import geopandas as gpd

data = gpd.read_file("../data/lfsa000b21a_e.shp")

In [3]:
data.head()

,CFSAUID,DGUID,PRUID,PRNAME,LANDAREA,geometry
0,A0A,2021A0011A0A,10,Newfoundland and Labrador / Terre-Neuve-et-Lab...,4136.6221,"MULTIPOLYGON (((8986398.603 2027249.146, 89863..."
1,A0B,2021A0011A0B,10,Newfoundland and Labrador / Terre-Neuve-et-Lab...,4152.6284,"MULTIPOLYGON (((8942820.023 2019517.649, 89428..."
2,A0C,2021A0011A0C,10,Newfoundland and Labrador / Terre-Neuve-et-Lab...,2733.4722,"MULTIPOLYGON (((8881604.92 2158976.151, 888160..."
3,A0E,2021A0011A0E,10,Newfoundland and Labrador / Terre-Neuve-et-Lab...,6365.8235,"MULTIPOLYGON (((8824262.857 1952006.689, 88242..."
4,A0G,2021A0011A0G,10,Newfoundland and Labrador / Terre-Neuve-et-Lab...,15499.6047,"MULTIPOLYGON (((8847567.966 2181504.514, 88475..."


In [4]:
data[data.CFSAUID=="L3J"]

,CFSAUID,DGUID,PRUID,PRNAME,LANDAREA,geometry


In [52]:
data_geometry = data.iloc[:,[0,5]]
data_geometry

,CFSAUID,geometry
0,A0A,"MULTIPOLYGON (((8986398.603 2027249.146, 89863..."
1,A0B,"MULTIPOLYGON (((8942820.023 2019517.649, 89428..."
2,A0C,"MULTIPOLYGON (((8881604.92 2158976.151, 888160..."
3,A0E,"MULTIPOLYGON (((8824262.857 1952006.689, 88242..."
4,A0G,"MULTIPOLYGON (((8847567.966 2181504.514, 88475..."
...,...,...
1638,X0G,"POLYGON ((4569793.423 3068648.757, 4567685.511..."
1639,X1A,"POLYGON ((5233904.58 3460235.34, 5284583.063 3..."
1640,Y0A,"POLYGON ((4463431.114 3683146.831, 4463181.651..."
1641,Y0B,"MULTIPOLYGON (((4089475.866 3398234.78, 408586..."


In [57]:
data_geometry.geometry.loc[data_geometry.CFSAUID == "H2X"]

405    POLYGON ((7632387.577 1245597.169, 7632358.786...
Name: geometry, dtype: geometry

In [1]:
import pandas as pd

In [2]:
data_postal= pd.read_csv("../data/ZipCodeFiles/CanadianPostalCodes202403.csv")

In [3]:
data_postal

,POSTAL_CODE,CITY,PROVINCE_ABBR,TIME_ZONE,LATITUDE,LONGITUDE
0,J9P 7B7,VAL-D'OR,QC,5,48.071162,-77.872634
1,M4C 1S9,TORONTO,ON,5,43.688438,-79.307762
2,T8L 0X6,FORT SASKATCHEWAN,AB,7,53.684795,-113.230031
3,N3H 1K9,CAMBRIDGE,ON,5,43.400606,-80.359684
4,M8W 2C9,ETOBICOKE,ON,5,43.605177,-79.534144
...,...,...,...,...,...,...
900683,K1Y 3G1,OTTAWA,ON,5,45.399437,-75.728286
900684,K2M 2E8,KANATA,ON,5,45.302607,-75.855760
900685,E5N 5X3,HAMPTON,NB,4,45.538400,-65.838020
900686,G1R 1P8,QUEBEC,QC,5,46.810471,-71.218814


In [4]:
data_postal[data_postal.POSTAL_CODE=="H2X 3Y3"]

,POSTAL_CODE,CITY,PROVINCE_ABBR,TIME_ZONE,LATITUDE,LONGITUDE
68646,H2X 3Y3,MONTREAL,QC,5,45.511795,-73.55933


In [5]:
import folium

# --- 1️⃣ Create base map (centered on Montreal) ---
m = folium.Map(location=[45.5019, -73.5674], zoom_start=12)

# --- 2️⃣ Add Mapbox Traffic layer ---
# ⚠️ You need to replace YOUR_MAPBOX_ACCESS_TOKEN with your token
mapbox_token = "pk.eyJ1IjoiYW1hZG91c25kaWF5ZSIsImEiOiJjbWdseGQ2MTYwbTRrMmxxNTgzMXNoajlrIn0.huvmnGzS1yjykOIDqMeOyw"

folium.TileLayer(
    tiles=f"https://api.mapbox.com/styles/v1/mapbox/traffic-night-v2/tiles/256/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_token}",
    attr='Mapbox Traffic',
    name='Traffic (Mapbox)',
    overlay=True,
    control=True
).add_to(m)

# --- 3️⃣ Add optional other base layers (for contrast) ---
folium.TileLayer('OpenStreetMap', name='OSM Base').add_to(m)
folium.TileLayer('cartodbpositron', name='Light Map').add_to(m)

# --- 4️⃣ Add layer control ---
folium.LayerControl().add_to(m)

# --- 5️⃣ Save map ---
m.save("montreal_traffic.html")
print("✅ Map saved as montreal_traffic.html")


✅ Map saved as montreal_traffic.html


In [10]:
import requests
import geopandas as gpd
from shapely.geometry import shape

# --- Config ---
MAPBOX_TOKEN = "pk.eyJ1IjoiYW1hZG91c25kaWF5ZSIsImEiOiJjbWdseGQ2MTYwbTRrMmxxNTgzMXNoajlrIn0.huvmnGzS1yjykOIDqMeOyw"
TILESET_ID = "mapbox.boundaries-postal-code-prefix-ca"

# Example coordinate in Montréal
lon, lat = -73.5673, 45.5017
url = (
    f"https://api.mapbox.com/v4/{TILESET_ID}/tilequery/{lon},{lat}.json"
    f"?access_token={MAPBOX_TOKEN}"
)

# --- Query Mapbox ---
response = requests.get(url)
data = response.json()
print(response.status_code)
print(data)


404
{'message': 'Tileset "mapbox.boundaries-postal-code-prefix-ca" does not exist'}


In [ ]:

# --- Convert to GeoDataFrame ---
features = []
for feature in data["features"]:
    geom = shape(feature["geometry"])
    props = feature["properties"]
    features.append({"geometry": geom, **props})

gdf = gpd.GeoDataFrame(features, crs="EPSG:4326")

print(gdf[["name", "iso_3166_2", "geometry"]].head())
